- 240621(금) 중앙대학교 군 장병 AISW 역량강화: 고급자연어처리 실습 자료입니다.
- 본 내용은 IIPL (Intelligent Information Processing Lab) 소속 석사과정 김영화 조교가 작성하였습니다.

---
## 03
- DPO (Direct Preference Optimization)
- [ref](https://devocean.sk.com/blog/techBoardDetail.do?ID=165903&boardType=techBlog#none)
---

## Setting
### 라이브러리 설치

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip install -q -U transformers==4.38.2
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U accelerate==0.27.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U trl==0.7.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.3 MB/s eta 0:00:00


## ChatML (Chat Markup Languague)

###  ChatML Prompt
- OpenAI에서 대화 인터페이스를 효과적으로 관리 할 수 있도록 데이터의 구조를 나타내는 구문
- ChatML prompt 형식
```json
<|im_start|>system
모델의 초기 지침 사항
<|im_end|>
<|im_start|>user
사용자의 메시지
<|im_end|>
<|im_start|>assistant
```
  - 문장의 시작은 <|im_start|> 로 시작, 이후 역할 (System, User, Assistant) 을 명시하고 문장의 끝은 <|im_end|> 구분자 토큰

  1. 시스템 메시지로 모델의 초기 지침 사항을 설명하며, 모델이 사용자의 질문에 어떻게 반응해야 할지에 대한 지침이나 규칙등을 명시.

  2. 사용자 메시지로 모델에게 질문할 내용

  3. 모델이 응답할 차례임을 나타내는 <|im_start|>assistant 토큰

## Gemma for TinyLlama
- https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.6
  - Chat template: Chat prompt 형식
  ```json
<|system|>
모델의 초기 지침 사항 </s>
<|user|>
사용자의 메시지 </s>
<|assistant|>
```

### Model Load

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

- special tokens 확인

In [33]:
print("Special Tokens:", tokenizer.special_tokens_map)

Special Tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}


### Chat 형식으로 대화

In [ ]:
question = "Why do I hate summer?"

In [ ]:
prompt = f"""<bos><start_of_turn>system
You are a helpful AI assistant.<end_of_turn>
<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""

In [ ]:
# 모델 출력
from transformers import pipeline

pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer, max_new_tokens=256)
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"])

### Chat template 사용
- HF tokenizer는 ChaML 형식을 만들 수 있는 템플릿을 제공
  - https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
chat = [
    { "role": "user", "content": "Which country's capital is Seoul?" },
    { "role": "assistant", "content": "Seoul is the capital of Korea." },
    { "role": "user", "content": "How many people live in Seoul?" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

### ChatML function
- multiturn 형식으로 chatbot 처럼 대화할 수 있도록 prompt를 만들어 주는 함수 생성

In [ ]:
messages = []

def chat_func(input):
    messages.append({"role": "user", "content": input})
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    print("prompt:", prompt)

    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=256)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    decoded_output = decoded_output.replace("<eos>", "").strip()
    parts = decoded_output.split('<start_of_turn>model')
    last_output = parts[-1]
    print(last_output)

    messages.append({"role": "assistant", "content": last_output})

In [ ]:
# 확인 코드
chat_func("Why do I like summer?")

In [ ]:
chat_func("How can you feel the beauty of nature?")

## DPO

## Dataset Load
- https://huggingface.co/datasets/jondurbin/truthy-dpo-v0.1

In [18]:
from datasets import load_dataset

dataset = load_dataset("jondurbin/truthy-dpo-v0.1")

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 1016
    })
})

In [ ]:
dataset['train'][200]

### 학습용 프롬프트 조정
- TinyLlama special token을 통해 chatml 형식으로 만듦

In [4]:
def generate_prompt(example):
    prompt = example['prompt']
    rejected = example['rejected']
    chosen = example['chosen']

    example['prompt'] = f"<bos><start_of_turn>system\n <end_of_turn><start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    example['rejected'] = f"{rejected}<end_of_turn>\n<eos>"
    example['chosen'] = f"{chosen}<end_of_turn>\n<eos>"

    return example

In [5]:
transformed_dataset = dataset.map(generate_prompt)

Map:   0%|          | 0/1016 [00:00<?, ? examples/s]

In [ ]:
# 확인
transformed_dataset['train'][0]

In [28]:
# data split
dataset = transformed_dataset['train'].train_test_split(test_size=0.05)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 965
    })
    test: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 51
    })
})

### DPO 학습
-  Colab에서 학습 할 수 있도록 QLoRa를 활용해 모델을 올리고 DPO 학습을 진행

In [8]:
import torch
from transformers import BitsAndBytesConfig
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# 모델 load (앞에서 해서 생략)
# BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"
# base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

- DPO Trainer 실행

In [10]:
# Training Arguments 설정
from transformers import TrainingArguments

training_args = TrainingArguments(
        output_dir="./outputs",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_32bit",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        per_device_eval_batch_size=2,
        logging_steps=100,
        learning_rate=5e-7,
        eval_steps=100,
        num_train_epochs=1,
        save_steps=500,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
)

In [14]:
from trl import DPOTrainer

trainer = DPOTrainer(
    base_model,
    ref_model=None,
    args=training_args,
    beta=0.1,
    peft_config=lora_config,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    max_prompt_length=512,
    max_length=1024,
)

Map:   0%|          | 0/965 [00:00<?, ? examples/s]

In [15]:
# 학습: 약 25분
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
100,0.692800,0.691823,0.002456,-0.000231,0.903846,0.002687,-187.987686,-148.512985,-3.986944,-3.745275
200,0.689700,0.687995,0.011727,0.001134,0.942308,0.010593,-187.974030,-148.420273,-3.987164,-3.745259
300,0.686500,0.684724,0.020212,0.002858,0.961538,0.017354,-187.956787,-148.335419,-3.987332,-3.745181
400,0.682600,0.681567,0.028859,0.004969,0.942308,0.023889,-187.935684,-148.248962,-3.987494,-3.745112


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
100,0.692800,0.691823,0.002456,-0.000231,0.903846,0.002687,-187.987686,-148.512985,-3.986944,-3.745275
200,0.689700,0.687995,0.011727,0.001134,0.942308,0.010593,-187.974030,-148.420273,-3.987164,-3.745259
300,0.686500,0.684724,0.020212,0.002858,0.961538,0.017354,-187.956787,-148.335419,-3.987332,-3.745181
400,0.682600,0.681567,0.028859,0.004969,0.942308,0.023889,-187.935684,-148.248962,-3.987494,-3.745112
500,0.679200,0.678941,0.034989,0.005625,0.923077,0.029364,-187.929123,-148.187653,-3.987673,-3.745144
600,0.678200,0.677107,0.039988,0.006779,0.942308,0.033209,-187.917572,-148.137665,-3.987791,-3.745154
700,0.674000,0.675678,0.043408,0.007175,0.942308,0.036233,-187.913605,-148.103455,-3.987892,-3.745169
800,0.678300,0.675086,0.044787,0.007303,0.942308,0.037484,-187.912354,-148.089676,-3.987914,-3.745158
900,0.675100,0.674908,0.045150,0.007291,0.942308,0.037859,-187.912460,-148.086044,-3.987925,-3.745157


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=965, training_loss=0.6811972702105429, metrics={'train_runtime': 1245.076, 'train_samples_per_second': 0.775, 'train_steps_per_second': 0.775, 'total_flos': 0.0, 'train_loss': 0.6811972702105429, 'epoch': 1.0})

### 학습된 lora weight 저장

In [16]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


- 하나의 fine-tuned model

In [20]:
from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained("./TinyLlama-1.1B-Chat-v0.6_DPO") # Save the merged model

### 추론

- 저장한 모델 로드

In [34]:
# GPU memory 이슈로 로드 주석

# BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"
# FINETUNE_MODEL = "./TinyLlama-1.1B-Chat-v0.6_DPO"

# base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
# finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [35]:
from transformers import pipeline
pipe_base = pipeline("text-generation", model=base_model, tokenizer=tokenizer, max_new_tokens=512)

In [26]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [29]:
prompt = dataset['test'][10]['prompt']

In [30]:
prompt

'<bos><start_of_turn>system\n <end_of_turn><start_of_turn>user\nDo wooden pencils contain lead as their core?<end_of_turn>\n<start_of_turn>model\n'

- DPO 이전 모델 추론

In [31]:
outputs = pipe_base(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"])

<bos><start_of_turn>system
 <end_of_turn><start_of_turn>user
Do wooden pencils contain lead as their core?<end_of_turn>
<start_of_turn>model
Yes, they do. The core of a wooden pencil is made from the wood used to make it.


- DPO 이후 모델 추론

In [32]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"])

<bos><start_of_turn>system
 <end_of_turn><start_of_turn>user
Do wooden pencils contain lead as their core?<end_of_turn>
<start_of_turn>model
Wooden pencils do not contain lead.
